In [ ]:
!nvidia-smi

Fri Sep 13 11:32:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%writefile exemplo1.cu
#include <thrust/device_vector.h> // Inclui a biblioteca para vetores na GPU (device)
#include <thrust/host_vector.h>   // Inclui a biblioteca para vetores na CPU (host)
#include <iostream>               // Biblioteca padrão para entrada e saída de dados

int main() {
    // Cria um vetor na CPU (host) com 5 elementos, todos inicializados com 0
    thrust::host_vector<double> host(5, 0);
    host[4] = 35; // Altera o último elemento do vetor para 35

    /* Os dados do vetor 'host' são copiados para a GPU,
       criando um vetor equivalente na GPU (device) */
    thrust::device_vector<double> dev(host);

    /* Altera o vetor na CPU, mas não afeta o vetor na GPU,
       pois a cópia já foi feita anteriormente */
    host[2] = 12; // Altera o terceiro elemento do vetor na CPU para 12

    // Exibe os elementos do vetor na CPU
    printf("Host vector: ");
    for (auto i = host.begin(); i != host.end(); i++) {
        std::cout << *i << " "; // Acesso rápido aos elementos na CPU
    }
    printf("\n");

    // Exibe os elementos do vetor na GPU
    printf("Device vector: ");
    for (auto i = dev.begin(); i != dev.end(); i++) {
        std::cout << *i << " "; // Acesso aos elementos na GPU é lento, pois os dados estão na GPU
    }
    printf("\n");
}

Overwriting exemplo1.cu


In [ ]:
!nvcc -arch=sm_75 -std=c++14 exemplo1.cu -o exemplo1

In [ ]:
!./exemplo1

Host vector: 0 0 12 0 35 
Device vector: 0 0 0 0 35 


In [ ]:
%%writefile exemplo2.cu
#include <thrust/device_vector.h>              // Inclui a biblioteca para vetores na GPU
#include <thrust/host_vector.h>                // Inclui a biblioteca para vetores na CPU
#include <thrust/sequence.h>                   // Inclui a função para preencher vetores com uma sequência de números
#include <thrust/functional.h>                 // Inclui operações matemáticas padrão, como soma e multiplicação
#include <thrust/transform.h>                  // Inclui a função para transformar vetores com operações ponto a ponto
#include <thrust/iterator/constant_iterator.h> // Inclui o iterador constante necessário
#include <iostream>

int main() {
    // Cria um vetor na GPU (device_vector) com 10 elementos, todos inicializados com 0
    thrust::device_vector<double> V1(10, 0);
    // Preenche V1 com uma sequência de números: {0, 1, 2, ..., 9}
    thrust::sequence(V1.begin(), V1.end());

    // Cria um vetor na GPU com 5 elementos, todos inicializados com 0
    thrust::device_vector<double> V2(5, 0);
    // Preenche os dois primeiros elementos de V2 com 5.5: {5.5, 5.5, 0, 0, 0}
    thrust::fill(V2.begin(), V2.begin() + 2, 5.5);
    // Preenche os elementos restantes de V2 com 10: {5.5, 5.5, 10, 10, 10}
    thrust::fill(V2.begin() + 2, V2.end(), 10);

    // Cria dois vetores na GPU com 10 elementos, para armazenar resultados das operações
    thrust::device_vector<double> V3(10); // Vetor para armazenar o resultado da soma
    thrust::device_vector<double> V4(10); // Vetor para armazenar o resultado da multiplicação

    // Aplica a operação de soma elemento por elemento entre V1 e V2 e armazena o resultado em V3
    // Como V2 tem menos elementos, o restante de V1 é somado com zeros (elemento padrão).
    // Resultado: V3 = {0+5.5, 1+5.5, 2+10, 3+10, 4+10, 5+0, ..., 9+0}
    thrust::transform(V1.begin(), V1.end(), V2.begin(), V3.begin(), thrust::plus<double>());

    // Aplica multiplicação de V1 com o valor constante 0.5
    // e armazenaria o resultado em V4.
    thrust::transform(V1.begin(), V1.end(), thrust::constant_iterator<double>(0.5), V4.begin(), thrust::multiplies<double>());

    // Imprime os elementos de V1
    printf("V1: ");
    for (thrust::device_vector<double>::iterator i = V1.begin(); i != V1.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V1
    }
    printf("\n");

    // Imprime os elementos de V2
    printf("V2: ");
    for (thrust::device_vector<double>::iterator i = V2.begin(); i != V2.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V2
    }
    printf("\n");

    // Imprime os elementos de V3
    printf("V3: ");
    for (thrust::device_vector<double>::iterator i = V3.begin(); i != V3.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V3
    }
    printf("\n");

    // Imprime o vetor V4, que foi criado mas não utilizado.
     printf("V4: ");
    for (thrust::device_vector<double>::iterator i = V4.begin(); i != V4.end(); i++) {
         std::cout << *i << " ";
    }
    printf("\n");

    return 0;
}

Writing exemplo2.cu


In [ ]:
!nvcc -arch=sm_75 -std=c++14 exemplo2.cu -o exemplo2

In [ ]:
!./exemplo2

V1: 0 1 2 3 4 5 6 7 8 9 
V2: 5.5 5.5 10 10 10 
V3: 5.5 6.5 12 13 14 5 6 7 8 9 
V4: 0 0.5 1 1.5 2 2.5 3 3.5 4 4.5 


In [ ]:
%%writefile stocks.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main() {
    int n = 2518;                   // Define o tamanho do vetor (2518 elementos)
    double value = 0.0;             // Variável auxiliar para armazenar o valor lido da entrada
    std::chrono::duration<double> diff; // Variável para armazenar a diferença de tempo calculada

    // Marca o início da medição do tempo de leitura dos dados
    auto leitura_i = std::chrono::steady_clock::now();

    // Cria um vetor na CPU (host_vector) com n elementos, todos inicializados com 0
    thrust::host_vector<double> host(n, 0);

    // Loop para ler n valores da entrada padrão e armazená-los no vetor host
    for (int i = 0; i < n; i++) {
        cin >> value;              // Lê um valor da entrada padrão
        host[i] = value;           // Armazena o valor lido no vetor host
    }

    // Marca o fim da medição do tempo de leitura dos dados
    auto leitura_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na leitura dos dados
    diff = leitura_f - leitura_i;

    // Exibe o tempo de leitura em segundos
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;

    // Marca o início da medição do tempo de cópia dos dados da CPU para a GPU
    auto copia_i = std::chrono::steady_clock::now();

    // Cria um vetor na GPU (device_vector) copiando os dados do vetor host da CPU
    thrust::device_vector<double> dev(host);

    // Marca o fim da medição do tempo de cópia dos dados
    auto copia_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na cópia dos dados para a GPU
    diff = copia_f - copia_i;

    // Exibe o tempo de cópia em segundos
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;

    return 0;
}


Writing stocks.cu


In [ ]:
!nvcc -arch=sm_70 -std=c++14 stocks.cu -o stocks

In [1]:
!./stocks < stocks-google.txt

/bin/bash: line 1: stocks-google.txt: No such file or directory


In [61]:
%%writefile stocks-google.txt
1015.24
1020.50
1022.00
1018.75
1025.00
1028.50
1031.75
1030.00
1035.00
1032.50
1038.00
1032.75
1039.00
1040.50
1043.00
1045.25
1040.00
1038.50
1042.25
1047.75
1049.00
1051.50
1054.00
1058.25
1060.50
1065.75
1068.00
1070.50
1073.25
1075.75
1078.00
1080.25
1082.50
1086.00
1088.25
1090.75
1093.00
1096.50
1098.75
1101.00
1105.50
1108.75
1110.00
1113.25
1116.50
1118.75
1122.00
1124.50
1127.00
1130.25
1132.75
1135.50
1138.00
1140.25
1143.00
1145.50
1148.00
1150.25
1153.75
1156.00
1158.50
1160.75
1164.25
1167.50
1170.00
1173.25
1176.00
1178.75
1182.00
1185.25
1188.00
1191.50
1194.00
1196.50
1198.75
1202.00
1204.25
1208.50
1211.00
1214.75
1218.00
1220.25
1223.50
1226.00
1229.25
1232.50
1235.00
1238.75
1241.00
1243.50
1246.00
1249.25
1251.50
1255.00
1258.25
1261.00
1264.75
1268.50
1271.00
1274.25
1277.75
1280.00
1283.50
1286.75
1290.25
1293.00
1296.50
1300.00
1302.50
1305.75
1309.00
1312.50
1315.00
1318.25
1320.75
1324.00
1327.50
1330.00
1333.25
1335.50
1338.75
1342.00
1345.25
1348.00
1350.50
1354.00
1356.75
1359.50
1362.25
1365.00
1368.75
1372.00
1375.50
1378.00
1381.25
1384.75
1387.50
1390.00
1393.25
1396.50
1399.00
1402.75
1405.00
1408.50
1411.75
1415.25
1418.00
1421.50
1425.00
1428.25
1430.50
1433.75
1437.00
1440.25
1443.50
1446.00
1449.75
1453.00
1456.50
1460.00
1462.50
1465.75
1469.00
1472.25
1475.50
1478.00
1481.25
1484.75
1488.00
1491.50
1494.75
1497.50
1501.00
1504.25
1507.50
1510.75
1514.00
1516.50
1519.75
1522.00
1525.50
1529.00
1532.25
1535.50
1539.00
1542.25
1545.00
1548.75
1552.00
1555.50
1558.75
1561.50
1565.00
1568.25
1571.75
1574.00
1577.25
1580.50
1583.75
1587.00
1590.25
1593.50
1596.75
1600.00
1603.25
1606.50
1609.75
1613.00
1616.25
1619.50
1623.00
1626.25
1629.50
1632.75
1636.00
1639.50
1642.75
1645.00
1648.25
1652.00
1655.25
1658.50
1661.75
1665.00
1668.25
1671.50
1674.75
1678.00
1681.25
1684.50
1687.75
1690.00
1693.25
1696.50
1699.75
1703.00
1706.25
1709.50
1712.75
1716.00
1719.25
1722.50
1726.00
1729.25
1732.50
1735.75
1739.00
1742.50
1745.75
1749.00
1752.25
1755.50
1758.75
1762.00
1765.25
1768.50
1772.00
1775.25
1778.50
1781.75
1785.00
1788.25
1791.50
1794.75
1798.00
1801.25
1804.50
1807.75
1811.00
1814.25
1817.50
1820.75
1824.00
1827.25
1830.50
1833.75
1837.00
1840.25
1843.50
1846.75
1850.00
1853.25
1856.50
1859.75
1863.00
1866.25
1869.50
1872.75
1875.00
1878.25
1881.50
1884.75
1888.00
1891.25
1894.50
1897.75
1901.00
1904.25
1907.50
1910.75
1914.00
1917.25
1920.50
1923.75
1927.00
1930.25
1933.50
1936.75
1940.00
1943.25
1946.50
1949.75
1953.00
1956.25
1959.50
1962.75
1966.00
1969.25
1972.50
1975.75
1979.00
1982.25
1985.50
1988.75
1992.00
1995.25
1998.50
2001.75
2005.00
2008.25
2011.50
2014.75
2018.00
2021.25
2024.50
2027.75
2031.00
2034.25
2037.50
2040.75
2044.00
2047.25
2050.50
2053.75
2057.00
2060.25
2063.50
2066.75
2070.00
2073.25
2076.50
2079.75
2083.00
2086.25
2089.50
2092.75
2096.00
2099.25
2102.50
2105.75
2109.00
2112.25
2115.50
2118.75
2122.00
2125.25
2128.50
2131.75
2135.00
2138.25
2141.50
2144.75
2148.00
2151.25
2154.50
2157.75
2161.00
2164.25
2167.50
2170.75
2174.00
2177.25
2180.50
2183.75
2187.00
2190.25
2193.50
2196.75
2200.00


Overwriting stocks-google.txt


In [110]:
%%writefile google_stock.cu

#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/reduce.h>
#include <thrust/functional.h>
#include <iostream>
#include <chrono>
#include <vector>
#include <fstream>
#include <string>
#include <cmath>  // Para DBL_MAX

struct Item {
    int valor12;
    int valor1;
};

bool lerDadosDoArquivo(const std::string& nomeArquivo, std::vector<double>& valor1es) {
    std::ifstream arquivo(nomeArquivo);
    if (!arquivo.is_open()){
        std::cerr << "Erro ao abrir o arquivo: " << nomeArquivo << std::endl;
        return false;
    }

    double valor1;
    while (arquivo >> valor1) {
        valor1es.push_back(valor1);
    }

    arquivo.close();
    return true;
}

bool preenche_vec_cpu(std::vector<double>& valor1es, thrust::host_vector<double>& vec_cpu){
    if (!valor1es.empty()) {
        for(size_t i = 0; i < valor1es.size(); i++){
            vec_cpu[i] = valor1es[i];
        }
        return true;
    } else {
        std::cerr << "Nenhum valor1 foi lido do arquivo." << std::endl;
        return false;
    }
}

bool preenche_vec_cpu_to_gpu(thrust::host_vector<double>& vec_cpu, thrust::device_vector<double>& vec_gpu){
    if (!vec_cpu.empty()) {
        for(size_t i = 0; i < vec_cpu.size(); i++){
            vec_gpu[i] = vec_cpu[i];
        }
        return true;
    } else {
        std::cerr << "Nenhum valor1 foi lido do arquivo." << std::endl;
        return false;
    }
}

int main() {
    std::string nomeArquivo = "stocks-google.txt";
    std::vector<double> valor1es;

    std::cerr << "Nome do arquivo: " << nomeArquivo << std::endl;

    if (!lerDadosDoArquivo(nomeArquivo, valor1es)) {
        std::cerr << "Falha ao ler o arquivo: " << nomeArquivo << std::endl;
        return 1;
    }

    thrust::host_vector<double> vec_cpu(valor1es.size());
    thrust::device_vector<double> vec_gpu(valor1es.size());
    thrust::device_vector<double> vec_gpu2(valor1es.size());

    auto inicio = std::chrono::high_resolution_clock::now();
    preenche_vec_cpu(valor1es, vec_cpu);
    auto fim = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duracao = fim - inicio;
    std::cerr << "duração para cpu O(n): " << duracao.count() << " segundos" << std::endl;

    auto inicio2 = std::chrono::high_resolution_clock::now();
    preenche_vec_cpu_to_gpu(vec_cpu, vec_gpu);
    auto fim2 = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duracao2 = fim2 - inicio2;
    std::cerr << "duração para gpu O(n): " << duracao2.count() << " segundos" << std::endl;

    auto inicio3 = std::chrono::high_resolution_clock::now();
    vec_gpu2 = vec_cpu;
    auto fim3 = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duracao3 = fim3 - inicio3;
    std::cerr << "duração para gpu só cópia: " << duracao3.count() << " segundos" << std::endl;

    std::cerr << "size: " << vec_gpu.size() << std::endl;

    double soma = thrust::reduce(vec_gpu.begin(), vec_gpu.end(), 0.0, thrust::plus<double>());
    double media = soma / vec_gpu.size();
    std::cerr << "media " << media << std::endl;

    size_t tamanho = vec_gpu.size();
    size_t pontos_para_ultimo_ano = 39;  // Aproximado pela regra de três
    size_t come = tamanho > pontos_para_ultimo_ano ? tamanho - pontos_para_ultimo_ano : 0;

    double soma_ultimo_ano = thrust::reduce(vec_gpu.begin() + come, vec_gpu.end(), 0.0, thrust::plus<double>());
    double media_ultimo_ano = soma_ultimo_ano / pontos_para_ultimo_ano;

    std::cout << "Média aproximada dos últimos 365 dias: " << media_ultimo_ano << std::endl;

    double max_total = thrust::reduce(vec_gpu.begin(), vec_gpu.end(), -DBL_MAX, thrust::maximum<double>());
    double min_total = thrust::reduce(vec_gpu.begin(), vec_gpu.end(), DBL_MAX, thrust::minimum<double>());

    std::cout << "Maior valor no histórico total: " << max_total << std::endl;
    std::cout << "Menor valor no histórico total: " << min_total << std::endl;

    // Encontrar o maior e menor valor no último ano (aproximadamente 39 pontos)
    size_t tam = tamanho > pontos_para_ultimo_ano ? tamanho - pontos_para_ultimo_ano : 0;

    double max_ultimo_ano = thrust::reduce(vec_gpu.begin() + tam, vec_gpu.end(), -DBL_MAX, thrust::maximum<double>());
    double min_ultimo_ano = thrust::reduce(vec_gpu.begin() + tam, vec_gpu.end(), DBL_MAX, thrust::minimum<double>());

    std::cout << "Maior valor no último ano: " << max_ultimo_ano << std::endl;
    std::cout << "Menor valor no último ano: " << min_ultimo_ano << std::endl;

    return 0;
}


Overwriting google_stock.cu


In [111]:
!nvcc -arch=sm_70 -std=c++14 google_stock.cu -o google_stocks

In [112]:
!./google_stocks

Nome do arquivo: stocks-google.txt
duração para cpu O(n): 2.7181e-05 segundos
duração para gpu O(n): 0.00344375 segundos
duração para gpu só cópia: 2.7005e-05 segundos
size: 389
media 1580.08
Média aproximada dos últimos 365 dias: 2138.25
Maior valor no histórico total: 2200
Menor valor no histórico total: 1015.24
Maior valor no último ano: 2200
Menor valor no último ano: 2076.5


In [169]:
%%writefile apple_micro.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
#include <vector>
#include <fstream>
#include <string>
#include <sstream>
#include <cstdlib>
#include <ctime>
#include <algorithm>
#include <thrust/transform.h>       // Inclui a biblioteca Thrust para operações de transformação
#include <thrust/functional.h>      // Inclui a biblioteca Thrust para funções de transformação

struct Item {
    double apple;
    double micro;
};

bool lerDadosDoArquivo(const std::string& nomeArquivo, std::vector<double>& apples, std::vector<double>& micro) {
    std::ifstream arquivo(nomeArquivo);
    if (!arquivo.is_open()){
        std::cerr << "Erro ao abrir o arquivo: " << nomeArquivo << std::endl;
        return false;
    }

    std::string linha;
    while (std::getline(arquivo, linha)) {
        std::istringstream iss(linha);
        std::string valorStr;
        double apple, microVal;
        if (std::getline(iss, valorStr) && !valorStr.empty()) {
            try {
                apple = std::stod(valorStr.substr(0, valorStr.find(',')));
                microVal = std::stod(valorStr.substr(valorStr.find(',') + 1));
                apples.push_back(apple);
                micro.push_back(microVal);
            } catch (const std::invalid_argument&) {
                std::cerr << "Erro ao converter valor: " << valorStr << std::endl;
                continue;
            }
        } else {
            std::cerr << "Linha inválida: " << linha << std::endl;
        }
    }

    arquivo.close();
    return true;
}

struct SubtractFunctor {
    __device__ double operator()(const double& a, const double& b) const {
        return a - b;
    }
};

bool dif(thrust::device_vector<double>& vec_gpu1, thrust::device_vector<double>& vec_gpu2) {
    if (!vec_gpu1.empty() && !vec_gpu2.empty() && vec_gpu1.size() == vec_gpu2.size()) {
        thrust::device_vector<double> vec_def(vec_gpu1.size());
        thrust::transform(
            vec_gpu1.begin(), vec_gpu1.end(),
            vec_gpu2.begin(),
            vec_def.begin(),
            SubtractFunctor()
        );

        std::vector<double> result(vec_def.size());
        thrust::copy(vec_def.begin(), vec_def.end(), result.begin());

        double soma = thrust::reduce(vec_def.begin(), vec_def.end(), 0.0, thrust::plus<double>());

        // Calcular a média
        double media = soma / vec_def.size();

        std::cerr << "Média: " << media << std::endl;


//        for (size_t i = 0; i < result.size(); i++) {
//            std::cerr << "Resultado [" << i << "]: " << result[i] << std::endl;
//        }

        return true;
    } else {
        std::cerr << "Nenhum valor foi lido do arquivo ou tamanhos diferentes." << std::endl;
        return false;
    }
}

int main() {
    std::string nomeArquivo = "apple_micro_teste";
    std::vector<double> apples;
    std::vector<double> micro;

    std::cerr << "Nome do arquivo: " << nomeArquivo << std::endl;

    if (!lerDadosDoArquivo(nomeArquivo, apples, micro)) {
        std::cerr << "Falha ao ler o arquivo: " << nomeArquivo << std::endl;
        return 1;
    }

    if (apples.empty() || micro.empty()) {
        std::cerr << "Nenhum dado foi carregado." << std::endl;
        return 1;
    }



    thrust::device_vector<double> vec_gpu(apples.size());
    thrust::device_vector<double> vec_gpu2(micro.size());

    thrust::copy(apples.begin(), apples.end(), vec_gpu.begin());
    thrust::copy(micro.begin(), micro.end(), vec_gpu2.begin());

    dif(vec_gpu, vec_gpu2);



    return 0;
}



Overwriting apple_micro.cu


In [170]:
!nvcc -arch=sm_70 -std=c++14 apple_micro.cu -o apple_micro

In [171]:
!./apple_micro

Nome do arquivo: apple_micro_teste
Média: -265.232
